In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sqlfunctions import get_datapunten_voor_model


## Notities
Datapunten AI:
- omzet
- Beursgenoteerd of niet
- sector (nacebel code)
- aantal personeelsleden
- hoofdsteden provincies

In [9]:
# Zet alle data in pd dataframe
dataframe = pd.DataFrame(columns=['ondernemingnr', 'omzet' ,'beursgenoteerd', 'sector', 'personeelsleden', 'hoofdsteden_provincies', 'score'])
dataframe.set_index('ondernemingnr', inplace=True)
# TODO: 
# - Hoe kom ik aan hoofdsteden_provincies?
# - De scores van alle bedrijven moet nog berekend worden zodat ik een Y heb
# - 'site_aanwezig', 'pdf_aanwezig' -> later toevoegen en kijken of dit de accuracy verbeterd
data = get_datapunten_voor_model()

In [12]:
dataframe

,omzet,beursgenoteerd,sector,personeelsleden,hoofdsteden_provincies,score
ondernemingnr,,,,,,


In [13]:
# add data to dataframe
for ondernemingnr, omzet, beursgenoteerd, sector, personeelsleden, hoofdsteden, score in data:
    dataframe.loc[ondernemingnr] = [omzet, beursgenoteerd, sector, personeelsleden, hoofdsteden.rstrip(), score]
    # todo: Dit duurt anderhalve minuut, mss nog wat sneller maken ookal moet dat maar 1 keer gerund worden

In [14]:
dataframe

,omzet,beursgenoteerd,sector,personeelsleden,hoofdsteden_provincies,score
ondernemingnr,,,,,,
0538668417,40459907.0,False,Ontwerpen en programmeren van computerprogramma's,74.0,Dendermonde,0.006667
0426099818,39425216.0,False,Vervaardiging van voeders voor huisdieren,48.0,Antwerpen,0.000000
0884331578,41589842.0,False,Vervaardiging van ijzer en staal en van ferrol...,40.0,Gent,0.000000
0420305354,41764838.0,False,Detailhandel in auto's en lichte bestelwagens ...,37.0,Eeklo,0.000000
0404678357,38505882.0,False,Algemene bouw van andere niet-residentiële geb...,46.0,Antwerpen,0.000000
...,...,...,...,...,...,...
0888474369,748822.0,False,Algemene bouw van residentiële gebouwen,9.0,Tielt,0.000000
0890933518,1179982.0,False,Elektrotechnische installatiewerken aan gebouwen,17.0,Leuven,0.000000
0891334681,1114897.0,False,Algemene bouw van residentiële gebouwen,18.0,Gent,0.000000


In [20]:
# categorize sector
dataframe["sector"] = dataframe["sector"].astype('category')
# get amount of different sectors
sectors = dataframe["sector"].cat.categories
# how are the sectors distributed ?
dataframe["sector"].value_counts()


Goederenvervoer over de weg, m.u.v. verhuisbedrijven                             541
Detailhandel in auto's en lichte bestelwagens (kleiner of gelijk aan 3,5 ton)    355
Ontwerpen en programmeren van computerprogramma's                                302
Algemene bouw van residentiële gebouwen                                          264
Schrijnwerk                                                                      236
                                                                                ... 
Detailhandel in souvenirs en religieuze artikelen in gespecialiseerde winkels      1
Provinciale overheid                                                               1
Productie van televisiefilms                                                       1
Exploitatie van kermisattracties                                                   1
Verhuur en lease van caravans en motorhomes                                        1
Name: sector, Length: 744, dtype: int64

In [27]:
# Data preprocessing
dataframe["sector"] = dataframe["sector"].astype('category')

dataframe["beursgenoteerd"] = dataframe["beursgenoteerd"].astype(int)
len(dataframe[dataframe["beursgenoteerd"] == 1])
# Maar 17 beursgenoteerde bedrijven

dataframe["personeelsleden"] = dataframe["personeelsleden"].astype(int)


In [26]:
dataframe["omzet"].value_counts()

0.0    13460
Name: omzet, dtype: int64

![white box ml modellen](https://drek4537l1klr.cloudfront.net/thampi/HighResolutionFigures/figure_2-2.png)


In [28]:
# Model schrijven
# ! Kan pas gerund worden vanaf we alle data hebben
X = dataframe.drop(columns=["score", "ondernemingnr"])
y = dataframe["score"]

# Split data into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Decision tree regressor
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(random_state=42)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
accuracy_score(y_test, y_pred)

# Logistic regression
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

print(f"Accuracy Decision Tree: {accuracy_score(y_test, y_pred)} \n Accuracy Logistic Regression: {accuracy_score(y_test, y_pred)}")



,omzet,sector,personeelsleden,hoofdsteden_provincies
ondernemingnr,,,,
0404935507,0.0,Groothandel in diamant en andere edelstenen,15,None
0412527142,0.0,Overige vervoerondersteunende activiteiten,10,None
0461054460,0.0,"Groothandel in vaste, vloeibare en gasvormige ...",22,None
0632455341,0.0,"Groothandel in vaste, vloeibare en gasvormige ...",9,None
0475849633,0.0,Overige adviesbureaus op het gebied van bedrij...,10,None
...,...,...,...,...
0899773285,0.0,Chapewerken,9,None
0899789222,0.0,Ingenieurs en aanverwante technische adviseurs...,27,None
0899886915,0.0,Dakwerkzaamheden,10,None
